In [40]:
import pandas as pd
import plotly.express as px
import datetime

# 경쟁사

In [362]:
df = pd.read_csv('./datasets/경쟁사.csv')

- 별점 숫자로 바꾸기

In [364]:
def star(x):
    if x == 'star star-rate5':
        return 5
    elif x == 'star star-rate4':
        return 4
    elif x == 'star star-rate3':
        return 3        
    elif x == 'star star-rate2':
        return 2
    else:
        return 1

In [365]:
df['star'] = df['star'].apply(lambda x: star(x))

- 날짜를 datetime으로 변경

In [367]:
df['date'] = df['date'].apply(lambda x: x.split('.'))

In [368]:
df['date'] = df['date'].apply(lambda x: '-'.join(x))

In [369]:
def dtime(x):
    return datetime.datetime.strptime(x, '%Y-%m-%d')

In [370]:
df['date'] = df['date'].apply(lambda x: dtime(x))

In [371]:
dfd = df.copy()

- 분기별로 분류

In [374]:
def quarter(x):
    if x.month >= 10: 
        quart = '4/4'
    elif x.month >= 7:
        quart = '3/4'
    elif x.month >= 4:
        quart = '2/4'
    elif x.month >= 1 :
        quart = '1/4'
    
    return (str(x.year) + ' ' + quart)

In [375]:
dfd['quarter'] = dfd['date'].apply(lambda x: quarter(x))

In [438]:
dfd.groupby('quarter').size()

quarter
2018 1/4    126
2018 2/4    254
2018 3/4    274
2018 4/4    298
2019 1/4    108
2019 2/4    196
2019 3/4    218
2019 4/4    245
2020 1/4    237
2020 2/4    353
2020 3/4    203
2020 4/4    291
2021 1/4    114
2021 2/4    259
2021 3/4    227
2021 4/4    309
2022 1/4    138
2022 2/4    192
2022 3/4    213
2022 4/4    306
2023 1/4    111
2023 2/4    169
2023 3/4     50
dtype: int64

In [384]:
dfqe = dfd.groupby(['quarter','emotion']).size().unstack().fillna(0)

- 부정 후기 비율을 ratio 컬럼에 저장

In [385]:
dfqe['ratio'] = (dfqe[False]/(dfqe[True]+dfqe[False]))*100

- 총 리뷰 개수를 리뷰 개수 컬럼에 저장

In [442]:
dfqe['리뷰 개수'] = dfd.groupby('quarter').size()

In [444]:
dfqe2 = dfqe.reset_index()

In [445]:
dfqe2

emotion,quarter,False,True,ratio,리뷰 개수
0,2018 1/4,2.0,124.0,1.587302,126
1,2018 2/4,7.0,247.0,2.755906,254
2,2018 3/4,3.0,271.0,1.094891,274
3,2018 4/4,11.0,287.0,3.691275,298
4,2019 1/4,0.0,108.0,0.000000,108
5,2019 2/4,11.0,185.0,5.612245,196
6,2019 3/4,2.0,216.0,0.917431,218
7,2019 4/4,7.0,238.0,2.857143,245
8,2020 1/4,2.0,235.0,0.843882,237
9,2020 2/4,2.0,351.0,0.566572,353


- 그래프 그려보기

In [389]:
import plotly.express as px
px.line(dfqe2, x = 'quarter', y = 'ratio',
       labels={'quarter':'분기', 'ratio':'부정 리뷰 비율(%)'})

## 경쟁사 별점

- 분기별 별점 개수로 정리

In [391]:
dfr = dfd.groupby(['quarter','star']).size().unstack().fillna(0)

In [392]:
dfr.head()

star,1,2,3,4,5
quarter,,,,,
2018 1/4,0.0,1.0,1.0,9.0,115.0
2018 2/4,0.0,2.0,7.0,20.0,225.0
2018 3/4,0.0,0.0,8.0,19.0,247.0
2018 4/4,2.0,1.0,9.0,18.0,268.0
2019 1/4,0.0,0.0,2.0,9.0,97.0


- 별점별 비율 구하기

In [393]:
dfrr = dfr.copy()

dfrr[1] = dfr[1]/(dfr[1]+dfr[2]+dfr[3]+dfr[4]+dfr[5])*100
dfrr[2] = dfr[2]/(dfr[1]+dfr[2]+dfr[3]+dfr[4]+dfr[5])*100
dfrr[3] = dfr[3]/(dfr[1]+dfr[2]+dfr[3]+dfr[4]+dfr[5])*100
dfrr[4] = dfr[4]/(dfr[1]+dfr[2]+dfr[3]+dfr[4]+dfr[5])*100
dfrr[5] = dfr[5]/(dfr[1]+dfr[2]+dfr[3]+dfr[4]+dfr[5])*100

In [394]:
dfrr['1+2+3'] = dfrr[1]+dfrr[2]+dfrr[3]
dfrr['1+2+3+4'] = dfrr[1]+dfrr[2]+dfrr[3]+dfrr[4]

In [396]:
dfrr2 = dfrr.reset_index()

In [397]:
dfrr2

star,quarter,1,2,3,4,5,1+2+3,1+2+3+4
0,2018 1/4,0.000000,0.793651,0.793651,7.142857,91.269841,1.587302,8.730159
1,2018 2/4,0.000000,0.787402,2.755906,7.874016,88.582677,3.543307,11.417323
2,2018 3/4,0.000000,0.000000,2.919708,6.934307,90.145985,2.919708,9.854015
3,2018 4/4,0.671141,0.335570,3.020134,6.040268,89.932886,4.026846,10.067114
4,2019 1/4,0.000000,0.000000,1.851852,8.333333,89.814815,1.851852,10.185185
5,2019 2/4,1.020408,0.000000,2.551020,7.142857,89.285714,3.571429,10.714286
6,2019 3/4,0.000000,0.000000,2.293578,5.504587,92.201835,2.293578,7.798165
7,2019 4/4,0.000000,0.000000,1.632653,7.755102,90.612245,1.632653,9.387755
8,2020 1/4,0.421941,0.000000,1.265823,5.063291,93.248945,1.687764,6.751055
9,2020 2/4,0.000000,0.283286,2.832861,5.382436,91.501416,3.116147,8.498584


In [398]:
import plotly.express as px
px.line(dfrr2, x = 'quarter', y = '1+2+3',
       labels={'quarter':'분기', '1+2+3':'별점 비율(1점~3점, %)'})

# T셀파
- 아이스크림과 같은 과정 반복

In [315]:
dff = pd.read_csv('sherpa_emot3.csv')

In [286]:
def star(x):
    if x == '5점':
        return 5
    elif x == '4점':
        return 4
    elif x == '3점':
        return 3        
    elif x == '2점':
        return 2
    else:
        return 1

In [287]:
dff['star'] = dff['star'].apply(lambda x: star(x))

In [325]:
dff['date'] = dff['date'].apply(lambda x: x.split('.'))

In [326]:
dff['date'] = dff['date'].apply(lambda x: '-'.join(x))

In [327]:
def dtime(x):
    return datetime.datetime.strptime(x, '%Y-%m-%d')

In [328]:
dff['date'] = dff['date'].apply(lambda x: dtime(x))

In [329]:
dffd = dff.copy()

In [330]:
str(dff['date'][0].year) + ' 1/4'

'2023 1/4'

In [331]:
dff['date'][0].month

7

In [332]:
def quarter(x):
    if x.month >= 10: 
        quart = '4/4'
    elif x.month >= 7:
        quart = '3/4'
    elif x.month >= 4:
        quart = '2/4'
    elif x.month >= 1 :
        quart = '1/4'
    
    return (str(x.year) + ' ' + quart)

In [333]:
dffd['quarter'] = dffd['date'].apply(lambda x: quarter(x))

In [439]:
dffd.groupby('quarter').size()

quarter
2018 1/4      4
2018 2/4     19
2018 3/4      9
2018 4/4     29
2019 1/4     14
2019 2/4     13
2019 3/4     13
2019 4/4     60
2020 1/4     25
2020 2/4     62
2020 3/4     33
2020 4/4     75
2021 1/4     38
2021 2/4    152
2021 3/4    198
2021 4/4    485
2022 1/4     88
2022 2/4    139
2022 3/4    273
2022 4/4    478
2023 1/4    110
2023 2/4    215
2023 3/4     63
dtype: int64

In [335]:
dffqe = dffd.groupby(['quarter','emotion_analysis']).size().unstack().fillna(0)

In [336]:
dffqe['ratio'] = (dffqe[False]/(dffqe[True]+dffqe[False]))*100

In [440]:
dffqe['리뷰 개수'] = dffd.groupby('quarter').size()

In [451]:
dffqe2 = dffqe.reset_index()

In [452]:
dffqe2

emotion_analysis,quarter,False,True,ratio,리뷰 개수
0,2018 1/4,0.0,4.0,0.000000,4
1,2018 2/4,4.0,15.0,21.052632,19
2,2018 3/4,0.0,9.0,0.000000,9
3,2018 4/4,1.0,28.0,3.448276,29
4,2019 1/4,1.0,13.0,7.142857,14
5,2019 2/4,2.0,11.0,15.384615,13
6,2019 3/4,2.0,11.0,15.384615,13
7,2019 4/4,3.0,57.0,5.000000,60
8,2020 1/4,1.0,24.0,4.000000,25
9,2020 2/4,4.0,58.0,6.451613,62


In [340]:
df.groupby('emotion_analysis').size()[0] / (df.groupby('emotion_analysis').size()[0] + df.groupby('emotion_analysis').size()[1]) * 100

1.8507918336195381

In [341]:
import plotly.express as px
px.line(dffqe2, x = 'quarter', y = 'ratio',
       labels={'quarter':'분기', 'ratio':'부정 리뷰 비율(%)'})

## T셀파 별점

In [343]:
dffr = dffd.groupby(['quarter','star']).size().unstack().fillna(0)

In [344]:
dffr.head()

star,1,2,3,4,5
quarter,,,,,
2018 1/4,0.0,0.0,0.0,0.0,4.0
2018 2/4,0.0,2.0,4.0,4.0,9.0
2018 3/4,0.0,0.0,1.0,1.0,7.0
2018 4/4,0.0,1.0,0.0,8.0,20.0
2019 1/4,0.0,1.0,0.0,4.0,9.0


In [345]:
dffrr = dffr.copy()

dffrr[1] = dffr[1]/(dffr[1]+dffr[2]+dffr[3]+dffr[4]+dffr[5])*100
dffrr[2] = dffr[2]/(dffr[1]+dffr[2]+dffr[3]+dffr[4]+dffr[5])*100
dffrr[3] = dffr[3]/(dffr[1]+dffr[2]+dffr[3]+dffr[4]+dffr[5])*100
dffrr[4] = dffr[4]/(dffr[1]+dffr[2]+dffr[3]+dffr[4]+dffr[5])*100
dffrr[5] = dffr[5]/(dffr[1]+dffr[2]+dffr[3]+dffr[4]+dffr[5])*100

In [346]:
dffrr['1+2+3'] = dffrr[1]+dffrr[2]+dffrr[3]
dffrr['1+2+3+4'] = dffrr[1]+dffrr[2]+dffrr[3]+dffrr[4]

In [348]:
dffrr2 = dffrr.reset_index()

In [349]:
dffrr2

star,quarter,1,2,3,4,5,1+2+3,1+2+3+4
0,2018 1/4,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000
1,2018 2/4,0.000000,10.526316,21.052632,21.052632,47.368421,31.578947,52.631579
2,2018 3/4,0.000000,0.000000,11.111111,11.111111,77.777778,11.111111,22.222222
3,2018 4/4,0.000000,3.448276,0.000000,27.586207,68.965517,3.448276,31.034483
4,2019 1/4,0.000000,7.142857,0.000000,28.571429,64.285714,7.142857,35.714286
5,2019 2/4,0.000000,15.384615,7.692308,15.384615,61.538462,23.076923,38.461538
6,2019 3/4,7.692308,0.000000,7.692308,15.384615,69.230769,15.384615,30.769231
7,2019 4/4,3.333333,1.666667,5.000000,13.333333,76.666667,10.000000,23.333333
8,2020 1/4,4.000000,4.000000,0.000000,4.000000,88.000000,8.000000,12.000000
9,2020 2/4,0.000000,1.612903,4.838710,20.967742,72.580645,6.451613,27.419355


In [350]:
import plotly.express as px
px.line(dffrr2, x = 'quarter', y = '1+2+3',
       labels={'quarter':'분기', '1+2+3':'별점 비율(1점~3점, %)'})

# 종합 그래프 그리기

In [469]:
# T셀파와 경쟁사 분류 추가
dfqe3 = dfqe2.copy()
dfqe3['분류'] = '경쟁사'

In [455]:
dffqe3 = dffqe2.copy()
dffqe3['분류'] = 'T셀파'

In [456]:
DF = pd.concat([dffqe3, dfqe3])

- 부정 리뷰 비율 그래프

In [436]:
import plotly.express as px
fig = px.line(DF, x = 'quarter', y = 'ratio', color = '분류',
       labels={'quarter':'분기', 'ratio':'부정 리뷰 비율(%)'})

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.865
))

fig.show()

In [421]:
dffrr3 = dffrr2.copy()
dffrr3['분류'] = 'T셀파'

In [423]:
dfrr3 = dfrr2.copy()
dfrr3['분류'] = '경쟁사'

In [426]:
DFs = pd.concat([dffrr3, dfrr3])

- 3점 이하 별점 비율 그래프

In [437]:
import plotly.express as px
fig = px.line(DFs, x = 'quarter', y = '1+2+3', color = '분류',
       labels={'quarter':'분기', '1+2+3':'별점 비율(1점~3점, %)'})

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.865
))

fig.show()

- 리뷰 개수 그래프

In [467]:
import plotly.express as px
fig = px.line(DF, x = 'quarter', y = '리뷰 개수', color = '분류',
       labels={'quarter':'분기','리뷰 개수':'리뷰 개수(T셀파: 개, 경쟁사: x10개)'})

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.005
))

fig.show()